In [1]:
import sys
sys.path.append('../../')
import os
os.chdir('../../')
# os.getcwd() # log absolute path to double check
%reload_ext autoreload
%autoreload 2

from datetime import datetime
import numpy as np
import torch
from datasets import get_dataset_motion_loader, get_motion_loader
from models import MotionTransformer
from utils.get_opt import get_opt
from utils.metrics import *
from datasets import EvaluatorModelWrapper
from collections import OrderedDict
from utils.plot_script import *
from utils import paramUtil
from utils.utils import *
from trainers import DDPMTrainer

from os.path import join as pjoin
import sys
from mask_model.train_transformer import TextEncoder
from mask_model.util import generate_samples, get_model

In [2]:
from mask_model.eval.evaluation import build_models, evaluation
mm_num_samples = 100 #2 # [INFO] from 100 
mm_num_repeats = 30 #2 # [INFO] from 30 
mm_num_times = 10 #1 # [INFO] from 10
diversity_times = 300
batch_size = 32
vq_path = '/home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale/'
transformer_path = '/home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/2023-03-10-10-22-39_5_transformer_textCond_fixZeroCond_13_vqgan_poseformer_1DisScale/'
opt_path = vq_path+'opt.txt'
# opt_path = '/home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/TEMP/opt.txt'
dataset_opt_path = opt_path

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gt_loader, gt_dataset = get_dataset_motion_loader(dataset_opt_path, batch_size, device)
wrapper_opt = get_opt(dataset_opt_path, device)
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

opt = get_opt(opt_path, device)
opt.debug = 't' # [INFO] mute w&b log
opt.gpu_id = list(eval(opt.gpu_id)) # [0] # 



Reading /home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale/opt.txt
Loading dataset kit ...


100%|██████████| 830/830 [00:00<00:00, 1722.17it/s]


Pointer Pointing at 0
Ground Truth Dataset Loading Completed!!!
Reading /home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!
Reading /home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/checkpoints/kit/2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale/opt.txt


In [3]:
from mask_model.mingpt import GPT
from mmcv.parallel import MMDistributedDataParallel, MMDataParallel
from mask_model.quantize import VectorQuantizer2
from mask_model.motiontransformer import MotionTransformerOnly2
generator_dim = 256
text_emb_dim = 512
codebook_dim = 32
dim_pose = opt.dim_pose
latent_dim = 256

# transformer
transformer = GPT(vocab_size=8192, block_size=196+int(text_emb_dim/generator_dim), 
                      n_layer=8, n_head=8, n_embd=generator_dim)
transformer.load_state_dict(torch.load(transformer_path+'transformer.pth'))
transformer = MMDataParallel(transformer.cuda(), device_ids=opt.gpu_id)
transformer.eval()

#
quantize = VectorQuantizer2(n_e = 8192, e_dim = codebook_dim)
quantize.load_state_dict(torch.load(vq_path+'/quantize.pth'))
quantize = MMDataParallel(quantize.cuda(), device_ids=opt.gpu_id)

decoder = MotionTransformerOnly2(input_feats=codebook_dim, 
                                output_feats=dim_pose, 
                                latent_dim=latent_dim, 
                                num_layers=8)
decoder.load_state_dict(torch.load(vq_path+'/decoder.pth'))
decoder = MMDataParallel(decoder.cuda(), device_ids=opt.gpu_id)

textEncoder = TextEncoder(output_dim=text_emb_dim).cuda()
textEncoder.load_state_dict(torch.load(transformer_path+'textEncoder.pth'))
# textEncoder = MMDataParallel(textEncoder.cuda(), device_ids=opt.gpu_id)


<All keys matched successfully>

In [8]:
class GenMotion:
    def __init__(self, transformer, quantize, decoder, textEncoder):
        print('GenMotion')
        self.encoder = transformer.eval()
        self.quantize = quantize.eval()
        self.decoder = decoder.eval()
        self.textEncoder = textEncoder.eval()

    def generate(self, caption, m_lens, dim_pose, batch_size=512):
        N = len(caption)
        cur_idx = 0
        self.encoder.eval()
        all_output = []
        while cur_idx < N:
            if cur_idx + batch_size >= N:
                batch_caption = caption[cur_idx:]
                batch_m_lens = m_lens[cur_idx:]
            else:
                batch_caption = caption[cur_idx: cur_idx + batch_size]
                batch_m_lens = m_lens[cur_idx: cur_idx + batch_size]
            output = self.generate_batch(batch_caption, batch_m_lens, dim_pose)
            B = output.shape[0]

            for i in range(B):
                all_output.append(output[i])
            cur_idx += batch_size
        return all_output

    def generate_batch(self, all_caption, all_m_lens, dim_pose):
        text_emb = self.textEncoder(all_caption, cond_mask_prob=0)
        text_emb = text_emb.view(text_emb.shape[0], -1, generator_dim)
        gen_indices = generate_samples(
            transformer, opt.device, text_emb, steps=opt.max_motion_length)
        x = get_model(self.quantize).embedding(gen_indices.reshape(-1))
        x = x.reshape(*gen_indices.shape, -1)
        x = self.decoder(x)
        # [TODO] steps=all_m_lens.max()
        # Filter unused length (to compair to gt)
        return x

    def load(*args):
        print('mocked load()')
        return 0, 0
    def eval_mode(*args):
        print('mocked eval_mode()')
    def to(*args):
        print('mocked to()')

trainer = GenMotion(transformer, quantize, decoder, textEncoder)


GenMotion


In [9]:
# for debug add this line to line 748 of /home/epinyoan/git/MaskText2Motion/MotionDiffuse/text2motion/models/gaussian_diffusion.py
# (under this::) indices = list(range(self.num_timesteps))[::-1]
# indices = indices[:4]
# encoder = build_models(opt, opt.dim_pose)
# trainer = DDPMTrainer(opt, encoder)

In [10]:
eval_motion_loaders = {
    'text2motion': lambda: get_motion_loader(
        opt,
        batch_size,
        trainer,
        gt_dataset,
        mm_num_samples,
        mm_num_repeats
    )
}

log_file = './t2m_evaluation.log'

In [11]:
evaluation(log_file, 
            gt_loader, 
            eval_wrapper,
            eval_motion_loaders, 
            mm_num_times, 
            diversity_times)

Generating 2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale ...
./checkpoints/kit/2023-03-04-20-39-40_13_vqgan_poseformer_1DisScale/model
mocked load()
mocked eval_mode()
mocked to()


734it [00:02, 256.54it/s]
100%|██████████| 196/196 [00:06<00:00, 31.58it/s]
734it [00:02, 283.77it/s]


Generated Dataset Loading Completed!!!
==================== Replication 0 ====================
Time: 2023-03-18 09:58:26.372594
========== Evaluating Matching Score ==========
---> [ground truth] Matching Score: 2.7452
---> [ground truth] R_precision: (top 1): 0.4148 (top 2): 0.6619 (top 3): 0.7898 
---> [text2motion] Matching Score: 8.0860
---> [text2motion] R_precision: (top 1): 0.0923 (top 2): 0.1591 (top 3): 0.2131 
Time: 2023-03-18 09:58:28.825404
========== Evaluating FID ==========
---> [ground truth] FID: 0.0210
---> [text2motion] FID: 33.7211
Time: 2023-03-18 09:58:30.129926
========== Evaluating Diversity ==========
---> [ground truth] Diversity: 10.5653
---> [text2motion] Diversity: 5.9319
Time: 2023-03-18 09:58:30.131924
========== Evaluating MultiModality ==========
---> [text2motion] Multimodality: 0.0000
!!! DONE !!!
========== Matching Score Summary ==========
---> [ground truth] Mean: 2.7452 CInterval: 0.0000
---> [text2motion] Mean: 8.0860 CInterval: 0.0000
==========

In [13]:
from datasets import build_dataloader
from datasets import Text2MotionDataset
from study.mylib import visualize_2motions
mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(opt.meta_dir, 'std.npy'))
train_split_file = pjoin(opt.data_root, 'train.txt')
train_dataset = Text2MotionDataset(opt, mean, std, train_split_file, times=50)
train_loader = build_dataloader(
    train_dataset,
    samples_per_gpu=opt.batch_size,
    drop_last=True,
    workers_per_gpu=4,
    shuffle=True,
    dist=opt.distributed,
    num_gpus=len(opt.gpu_id))

100%|██████████| 300/300 [00:00<00:00, 1945.89it/s]


In [15]:
caption, motions, m_lens = next(iter(train_loader))
# trainer.encoder.eval()
# text_emb = trainer.textEncoder(caption[-1:], cond_mask_prob=0)
# text_emb = text_emb.view(text_emb.shape[0], -1, generator_dim)
# gen_indices = generate_samples(transformer, opt.device, text_emb, steps=m_lens[-1])
# x = get_model(trainer.quantize).embedding(gen_indices.reshape(-1))
# x = x.reshape(*gen_indices.shape, -1)
# x = trainer.decoder(x)

text_emb = trainer.textEncoder(caption[-1:], cond_mask_prob=0)
text_emb = text_emb.view(text_emb.shape[0], -1, generator_dim)
gen_indices = generate_samples(trainer.encoder, opt.device, text_emb, steps=m_lens[-1])
x = get_model(trainer.quantize).embedding(gen_indices.reshape(-1))
x = x.reshape(*gen_indices.shape, -1)
x = trainer.decoder(x)

# x = trainer.generate_batch(caption[-2:], None, None)
x = x[0].detach().cpu().numpy()
visualize_2motions(x, std, mean, opt.dataset_name, x.shape[0])

condition_emb.shape[0]: 1


100%|██████████| 95/95 [00:01<00:00, 78.73it/s]


In [16]:
caption[-1:]

('A person touches the floor with their right hand and kneels down.',)